## A Logistic Regression Problem
### By MiMoTrix
#### Date: September 2018

In [79]:
import tensorflow as tf
import pandas as pd

##### Reads inputs from Titanic csv file using pandas

In [68]:
def inputs(file):
    data = pd.read_csv(file)
    data["Age"].fillna(0, inplace=True)
    data.head()
    data.replace({"Sex": {"male": 1, "female": 0}}, inplace=True)
    train_data = data.sample(frac=0.8, random_state=99)
    test_data = data.loc[~data.index.isin(train_data.index), :]
    X_train = train_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_train = pd.get_dummies(X_train, columns=['Pclass'])
    X_test = test_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_test = pd.get_dummies(X_test, columns=['Pclass'])
    Y_train = train_data.loc[:, ['Survived']]
    Y_test = test_data.loc[:, ['Survived']]
    return X_train, Y_train, X_test, Y_test
X_train, Y_train, X_test, Y_test = inputs("data/titanic.csv")
inputs("data/titanic.csv")

(     Sex    Age  Pclass_1  Pclass_2  Pclass_3
 247    0  24.00         0         1         0
 823    0  27.00         0         0         1
 566    1  19.00         0         0         1
 666    1  25.00         0         1         0
 199    0  24.00         0         1         0
 480    1   9.00         0         0         1
 86     1  16.00         0         0         1
 136    0  19.00         1         0         0
 253    1  30.00         0         0         1
 809    0  33.00         1         0         0
 208    0  16.00         0         0         1
 551    1  27.00         0         1         0
 78     1   0.83         0         1         0
 654    0  18.00         0         0         1
 48     1   0.00         0         0         1
 3      0  35.00         1         0         0
 453    1  49.00         1         0         0
 84     0  17.00         0         1         0
 514    1  24.00         0         0         1
 633    1   0.00         1         0         0
 326    1  61

##### Initializing weights and bias with zero

In [69]:
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")
print(W, b)

<tf.Variable 'weights_4:0' shape=(5, 1) dtype=float32_ref> <tf.Variable 'bias_4:0' shape=() dtype=float32_ref>


##### Combine Input and Weights and Bias

In [70]:
def matmulMaterials(X):
    return tf.matmul(tf.cast(X, tf.float32), W) + b

##### Predicting Y

In [71]:
def inference(X):
    return tf.sigmoid(matmulMaterials(X))

##### Loss function using Cross Entropy

In [49]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=matmulMaterials(X), labels=Y))

##### Train function with Gradient Decent Optimizer and 0.001 learning rate

In [50]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

##### Evaluating chance of survival of people ( > 0.5 will be chose as survived ) 

In [82]:
def evaluate(session, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print("Accuracy: ", session.run(tf.reduce_mean(tf.cast(tf.equal(predicted, tf.cast(Y, tf.float32)), tf.float32))))

##### MiniBatch Implemention

In [83]:
def miniBatch(size):
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        training_steps = 100
        for epoch in range(training_steps):
            for i in range(0, X_train.shape[0], size):
                session.run(train_op, feed_dict={x: X_train[i:i+size], y: Y_train[i:i+size]})
        evaluate(session, X_test, Y_test)
        session.close()
    return " "

MAIN

In [84]:
x = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]))
y = tf.placeholder(dtype=tf.float32, shape=(None, Y_train.shape[1]))
loss_func = loss(x, y)
train_op = train(loss_func)
import time
now = time.time()
print("Batch Size = 1", miniBatch(1))
print("Execution Time: ", time.time()-now)
now = time.time()
print("Batch Size = 5", miniBatch(5))
print("Execution Time: ", time.time()-now)
now = time.time()
print("Batch Size = 10", miniBatch(10))
print("Execution Time: ", time.time()-now)
now = time.time()
print("Batch Size = 100", miniBatch(100))
print("Execution Time: ", time.time()-now)

Accuracy:  0.78089887
Batch Size = 1  
Execution Time:  83.13866019248962


Accuracy:  0.48876405
Batch Size = 5  
Execution Time:  15.476040840148926


Accuracy:  0.5224719
Batch Size = 10  
Execution Time:  7.946897029876709


Accuracy:  0.64606744
Batch Size = 100  
Execution Time:  1.5575919151306152
